In [1]:
import math
import time
import os
import cv2 
import keyboard
import pickle
import numpy as np
from PIL import Image, ImageOps, ImageGrab
from matplotlib import pyplot as plt
from skimage import io, color, measure, morphology
from scipy import ndimage
from scipy.ndimage import rotate
import copy
%matplotlib inline

# Item detection
Scrap items appear to all appear at roughly the same shape, size, and colour. This suggests template matching as a solution. However, upon examining the training images, it was noted that each scrap item had the exact same colour albeit for a small difference observed in one case. This suggests that precise colour thresholding will be very effective at isolating the scrap items from the rest of the image. The steps for the item detection are as follows:

- RGB colour threeshold applied to the image to identify scrap items.
- Dilate segments of scrap items. This helps prevent map elements that sometimes break up the appearance of scrap as appearing as more than one scrap item when it is only one.

In [14]:
image = cv2.imread(r'training\3.png')

# Colour threshold for scrap items
scrap_lower_bound = (1, 254, 172)
scrap_upper_bound = (2, 255, 177)

# Colour threshold the image
color_mask = cv2.inRange(image, scrap_lower_bound, scrap_upper_bound)

# Dilate segments
kernel_size = (5, 5)
kernel = np.ones(kernel_size, np.uint8)
dilated_mask = cv2.dilate(color_mask, kernel, iterations=3)

cv2.imshow('Original Image', image)
cv2.imshow('Binary Mask', dilated_mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Enemy detection
Similar to scrap items, enemies all have the same shape and colour, but have differences in size. There are also issues with other objects appearing the same colour as the enemies on the monitor. This necessitates a requirement for circular hough transforms to identify enemies as opposed to locked doors or vents on the monitor. The following steps will be taken to identify enemies:

- RGB colour threshold applied to the image to filter out noise and any objects not red.
- Circular hough transform to identify circular segments.